# DIA stuff

Author: Melissa

Last updated: 2024-12-04

Let's try to pull data out of the APDB for DIA object, source, and forced source. This is related to Project 1 > Subproject 2 from this process document:

https://docs.google.com/document/d/1wEbmDjDT9XSby6KjG-1s7L8PG37nujf9sAV4HV2KEIY/edit?tab=t.0

Once the full set of data has been dumped to parquet, we will import into HATS in another notebook.


In [44]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import healpy as hp

import lsst.daf.butler as dafButler
from lsst.analysis.ap import apdb
from lsst.ap.association import AssociationTask, AssociationConfig
from lsst.dax.apdb import Apdb, ApdbCassandra, ApdbTables
import lsst.geom

from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, ManualInterval, AsinhStretch, MinMaxInterval, LogStretch


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from typing import TYPE_CHECKING, cast

import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import tqdm

plt.set_loglevel('WARNING')

Based on columns and query structure from Neven's notebook:

https://github.com/lsst-sitcom/notebooks_dia/blob/main/OR4/N_obj_src_truth_and_det.ipynb

In [38]:
columns = [
    "diaSourceId", 
    "diaObjectId", 
    "ra", 
    "dec", 
    "raErr", 
    "decErr", 
    "midpointMjdTai", 
    "psfFlux", 
    "psfFluxErr", 
    "scienceFlux", 
    "scienceFluxErr", 
    "snr", 
    "band", 
    "visit",
    "detector",    
    "x",
    "xErr",
    "y",
    "yErr",
    "time_processed", 
    "time_withdrawn",
    "isDipole",
    "centroid_flag",
    "apFlux_flag",
    "apFlux_flag_apertureTruncated",
    "psfFlux_flag",
    "psfFlux_flag_edge",
    "psfFlux_flag_noGoodPixels",
    "trail_flag_edge",
    "forced_PsfFlux_flag",
    "forced_PsfFlutx_flag_edge",
    "forced_PsfFlux_flag_noGoodPixels",
    "shape_flag",
    "shape_flag_no_pixels",
    "shape_flag_not_contained",
    "shape_flag_parent_source",
    "pixelFlags",
    "pixelFlags_bad",
    "pixelFlags_cr",
    "pixelFlags_crCenter",
    "pixelFlags_edge",
    "pixelFlags_interpolated",
    "pixelFlags_interpolatedCenter",
    "pixelFlags_offimage",
    "pixelFlags_saturated",
    "pixelFlags_saturatedCenter",
    "pixelFlags_suspect",
    "pixelFlags_suspectCenter",
    "pixelFlags_streak",
    "pixelFlags_streakCenter",
    "pixelFlags_injected",
    "pixelFlags_injectedCenter",
    "pixelFlags_injected_template",
    "pixelFlags_injected_templateCenter",
    "reliability"
]

# Convert list of columns into a comma-separated string
columns_string = ', '.join(f'"{col}"' for col in columns)

In [41]:

schema='jeremym_ppdb_replication_test_3'
instrument = 'LSSTComCamSim'
apdbQuery = apdb.ApdbPostgresQuery(instrument=instrument, namespace=schema)
# Then use this in your SQL query
with apdbQuery.connection as connection:
    for lower in range(0, 13_000_000, 1_000_000):
        src4_field = pd.read_sql_query(f'''
            SELECT *
            FROM "{schema}"."DiaSource"
            LIMIT 1000000
            offset {lower}
        ''', connection)
        src4_field.to_parquet(f"./DATA/dia/source/source_{lower}.parquet")
     

In [48]:
with apdbQuery.connection as connection:
    src4_field = pd.read_sql_query(f'''
        SELECT count(distinct("diaObjectId"))
        FROM "{schema}"."DiaObject"
    ''', connection)
src4_field
     

,count
0,12222552


In [45]:
with apdbQuery.connection as connection:
    for lower in tqdm(range(0, 13_000_000, 500_000)):
        src4_field = pd.read_sql_query(f'''
            SELECT *
            FROM "{schema}"."DiaObject"
            LIMIT 500000
            offset {lower}
        ''', connection)
        src4_field.to_parquet(f"./DATA/dia/object/object_{lower}.parquet")

100%|██████████| 26/26 [06:53<00:00, 15.89s/it]


In [40]:
# This is to see all of the columns avaliable
with apdbQuery.connection as connection:
    # Query to fetch all column names from the DiaObject table
    query_columns = '''
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'pp_ppdb_lsstcomcamsim_or4' AND table_name = 'DiaForcedSource';
    '''
    df_columns = pd.read_sql_query(query_columns, connection)
    print("Available columns:", df_columns['column_name'].tolist())

Available columns: ['diaForcedSourceId', 'diaObjectId', 'ra', 'dec', 'visit', 'detector', 'psfFlux', 'psfFluxErr', 'midpointMjdTai', 'scienceFlux', 'scienceFluxErr', 'band', 'time_processed', 'time_withdrawn']


In [ ]:
with apdbQuery.connection as connection:
    for lower in range(0, 70_000_000, 1_000_000):
        src4_field = pd.read_sql_query(f'''
            SELECT *
            FROM "{schema}"."DiaForcedSource"
            LIMIT 1000000
            offset {lower}
        ''', connection)
        src4_field.to_parquet(f"./DATA/dia/forced/forced_{lower}.parquet")